In [1]:
from deps.dobot_api import DobotApiDashboard, DobotApiMove
from deps import utils
from deps import cv_core
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pynput import keyboard
import time
import ipyparallel as ipp

In [2]:
rc = ipp.Cluster(n=1).start_and_connect_sync()
e0 = rc[0]
e0.block = False
e0.activate('0')

Starting 1 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>
100%|██████████| 1/1 [00:05<00:00,  5.52s/engine]


In [25]:
dash = DobotApiDashboard('192.168.1.6', 29999)
move = DobotApiMove('192.168.1.6', 30003)
#keys = utils.Keyboard(dash)

In [9]:
dash.ClearError()
dash.EnableRobot()

'0,{},EnableRobot();'

In [3]:
manmove = utils.ManualMove(move, dash)
manmove.execute()

In [3]:
utils.calib_pos(move)

'0,{},MovL(250.000000,0.000000,-90.000000,0.000000);'

In [19]:
prev=utils.get_pose(dash, verbose=False)

In [26]:
curr=utils.get_pose(dash, verbose=False)

In [27]:
x_off, y_off = (curr - prev)[:2]

In [22]:
%%px0
from deps import utils
from deps.dobot_api import DobotApiDashboard, DobotApiMove
dash = DobotApiDashboard('192.168.1.6', 29999)
move = DobotApiMove('192.168.1.6', 30003)
manmove = utils.ManualMove(move, dash)
manmove.execute()
coords = manmove.coords

<AsyncResult(%px): pending>

In [23]:
def draw_cross(w,h,frame):
    w = int(w)
    h = int(h)
    frame[h-5:h+6, w-1:w+2] = [0, 255, 0]
    frame[h-1:h+2, w-5:w+6] = [0, 255, 0]
    return frame

cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')

cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, cv_core.camera_res_dict['1200'])
cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', 1348, 1011)

coords = []

while(True):
    ret, frame = cap.read()
    h,w = frame.shape[:-1]
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)s

    # dst = cv2.cornerHarris(gray, blockSize=2, ksize=3, k=0.04)
    # frame[dst > 0.01 * dst.max()] = [0, 255, 0]
    frame = draw_cross(w/2, h/2, frame)
    frame = draw_cross(w/4, h/4, frame)
    frame = draw_cross(3*w/4, h/4, frame)
    frame = draw_cross(w/4, 3*h/4, frame)
    frame = draw_cross(3*w/4, 3*h/4, frame)
    cv2.imshow('frame',frame)

    k = cv2.waitKey(1)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@418.942] global /opt/conda/conda-bld/opencv-suite_1656606573658/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [5]:
ar = e0.pull('coords')
coords = ar.get()
coords

[array([256.062558, -17.195119, -90.033806, -17.456869]),
 array([297.919911, -17.01596 , -90.018532, -17.456888]),
 array([304.430722,  37.666581, -90.009521, -17.456944]),
 array([263.810049,  37.101424, -89.996956, -17.457005])]

In [6]:
w = 1600
h = 1200
xys = [(arr[0], arr[1]) for arr in coords]
robot_coor = utils.assign_corners(xys, reverse=True)
pix_coor = utils.assign_corners([(w/4, h/4), (3*w/4, h/4), (w/4, 3*h/4), (3*w/4, 3*h/4)])

features_mm_to_pixels_dict = {}
for key, value in robot_coor.items():
    features_mm_to_pixels_dict[value] = pix_coor[key]


tf_mtx = cv_core.compute_tf_mtx(features_mm_to_pixels_dict)

In [34]:
cont = cv_core.Contours()

def on_change(val): pass

offset = 40
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, cv_core.camera_res_dict['1200'])

cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')

cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', 1348, 1011)
cv2.createTrackbar('Manual Lock', 'frame', 0, 1, on_change)
cv2.createTrackbar('Mask Offset', 'frame', offset, 150, on_change)
cv2.setMouseCallback('frame', cont.mousecallback)

idx = 0
prev_point = (0,0,0)
while(True):
    ret, frame = cap.read()
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    val = cv2.getTrackbarPos('Manual Lock', 'frame')
    offset = cv2.getTrackbarPos('Mask Offset', 'frame')
    if val == 1:
        cont.locked = True
    else:
        cont.locked = False

    if cont.best_circ is None:
        while cont.best_circ is None:
            pt = cont.find_contours(frame, 10, offset)
    else:
        pt = cont.find_contours(frame, 10, offset)
        
    a,b,r = pt
    plot_img = frame.copy()
    cv2.circle(plot_img, (a, b), r, (3, 162, 255), 2)
    cv2.circle(plot_img, (a, b), 1, (0, 0, 255), 3)
    cv2.circle(plot_img, (a, b), r - offset, (0, 255, 0), 2)
    cv2.circle(plot_img, cont.big_circ[:2], cont.big_circ[2], (0, 0, 255), 2)
    cv2.putText(plot_img, f"{cont.big_circ[2]*2}px = 60mm", (cont.big_circ[0]-25, cont.big_circ[1] - cont.big_circ[2] - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    
    cv2.drawContours(plot_img, cont.singular, -1,(0,255, 0),2)
    cv2.drawContours(plot_img, cont.clusters, -1,(0,0,255),2)

    if cont.selected:
        cv2.drawContours(plot_img, cont.selected, -1,(255,0,0),2)

    for c in cont.singular:
        # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # draw the contour and center of the shape on the image
        #cv2.drawContours(plot_img, [c], -1, (0, 255, 0), 2)
        cv2.circle(plot_img, (cX, cY), 2, (0, 0, 255), -1)
        # cv2.putText(plot_img, f"{cX},{cY}", (cX - 20, cY - 20),
        # cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.putText(plot_img, f"{cv2.contourArea(c)}", (cX - 20, cY - 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1)
    #out.write(with_contours)

    for c in cont.clusters:
        # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # draw the contour and center of the shape on the image
        #cv2.drawContours(plot_img, [c], -1, (0, 255, 0), 2)
        cv2.circle(plot_img, (cX, cY), 2, (0, 0, 255), -1)
        # cv2.putText(plot_img, f"{cX},{cY}", (cX - 20, cY - 20),
        # cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)

        # cv2.putText(plot_img, f"{cv2.contourArea(c)}", (cX - 20, cY - 20),
        # cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1)


    if prev_point is pt:
        idx += 1
    else:
        idx = 0
    prev_point = pt


    if idx >= 30:
        message = 'LOCKED'
        color = (0,255,0)
    else:
        message = 'SEARCHING'
        color = (0,0,255)

    cv2.putText(plot_img, "TARGET:", (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)
    cv2.putText(plot_img, f"{message}", (125,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)
    cv2.putText(plot_img, f"Found: {len(cont.singular)}", (25,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)

    #cv2.setWindowProperty('frame', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow('frame',plot_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@706.009] global /opt/conda/conda-bld/opencv-suite_1656606573658/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [36]:
#centers_cuboids = cont.contour_centers(cont.singular)
centers_cuboids = cont.contour_centers(cont.selected)

dash.ClearError()
dash.EnableRobot()

#utils.default_pos(move)
for center in centers_cuboids:
    X, Y, _ = tf_mtx @ (center[0], center[1], 1)
    move.MovL(X+x_off, Y+y_off, -90,  0)
    time.sleep(1)
move.Sync()

#utils.default_pos(move)

'0,{},Sync();'

In [30]:
X, Y, _ = tf_mtx @ (679,785, 1)
move.MovL(X+x_off, Y+y_off, -90,  0)

'0,{},MovL(290.412517,1.630035,-90.000000,0.000000);'